# Weekly Execution Progress

Exact outstanding-order tracking from `wolfpack/targets.csv`.

Key metric:
- `% outstanding = abs(weekly_target_w - actual_w) / abs(weekly_target_w - start_w)`

This notebook plots both portfolio-level and symbol-level progress.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook

qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None


In [ ]:
df_targets = read_csv_from_store('wolfpack/targets.csv')
df_snapshots = read_csv_from_store('wolfpack/daily_snapshots.csv')

if df_targets is None:
    raise ValueError('targets.csv is required. Run a backtest after enabling target-state logging.')

df_targets['date'] = pd.to_datetime(df_targets['date'])
numeric_cols = ['start_w', 'weekly_target_w', 'scheduled_fraction', 'scheduled_w', 'actual_w', 'scale_day']
for col in numeric_cols:
    df_targets[col] = pd.to_numeric(df_targets[col], errors='coerce').fillna(0.0)

if df_snapshots is not None:
    df_snapshots['date'] = pd.to_datetime(df_snapshots['date'])

print(f'target rows: {len(df_targets):,}')
display(df_targets.head())


In [ ]:
df = df_targets.copy()
df['total_week_order_abs'] = (df['weekly_target_w'] - df['start_w']).abs()
df['remaining_order_abs'] = (df['weekly_target_w'] - df['actual_w']).abs()
df['remaining_order_abs'] = np.minimum(df['remaining_order_abs'], df['total_week_order_abs'])

df['outstanding_pct'] = np.where(
    df['total_week_order_abs'] > 1e-10,
    df['remaining_order_abs'] / df['total_week_order_abs'],
    0.0
)
df['filled_pct'] = 1.0 - df['outstanding_pct']

df = df.sort_values(['week_id', 'symbol', 'date'])
df['day_in_week'] = df.groupby(['week_id', 'symbol']).cumcount()

portfolio = (
    df.groupby(['date', 'week_id'], as_index=False)
      .agg(total_week_order_abs=('total_week_order_abs', 'sum'),
           remaining_order_abs=('remaining_order_abs', 'sum'))
      .sort_values('date')
)
portfolio['outstanding_pct'] = np.where(
    portfolio['total_week_order_abs'] > 1e-10,
    portfolio['remaining_order_abs'] / portfolio['total_week_order_abs'],
    0.0
)
portfolio['filled_pct'] = 1.0 - portfolio['outstanding_pct']
portfolio['day_in_week'] = portfolio.groupby('week_id').cumcount()

summary = (
    portfolio.groupby('week_id', as_index=False)
             .agg(week_total_order=('total_week_order_abs', 'max'),
                  final_outstanding_pct=('outstanding_pct', 'last'))
             .sort_values('week_id')
)
display(summary.tail(10))


In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(portfolio['date'], 100 * portfolio['outstanding_pct'], color='#1f77b4', linewidth=1.8)
ax.set_title('Portfolio Outstanding Weekly Order % (Daily)')
ax.set_ylabel('% outstanding')
ax.set_xlabel('Date')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

day_profile = (
    portfolio.groupby('day_in_week', as_index=False)['outstanding_pct']
             .agg(['mean', 'median'])
             .reset_index()
)
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(day_profile['day_in_week'], 100 * day_profile['mean'], marker='o', label='Mean')
ax.plot(day_profile['day_in_week'], 100 * day_profile['median'], marker='s', label='Median')
ax.set_title('Outstanding % by Day-In-Week')
ax.set_ylabel('% outstanding')
ax.set_xlabel('Scaling day index')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
symbol_order = (
    df.groupby('symbol', as_index=False)['total_week_order_abs']
      .mean()
      .sort_values('total_week_order_abs', ascending=False)
)
display(symbol_order.head(15))

selected_symbol = symbol_order['symbol'].iloc[0] if len(symbol_order) else None
if selected_symbol is not None:
    dfx = df[df['symbol'] == selected_symbol].copy().sort_values('date')
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(dfx['date'], 100 * dfx['outstanding_pct'], label='Outstanding %', color='#d62728')
    ax.plot(dfx['date'], 100 * dfx['scheduled_fraction'], label='Scheduled % of target', color='#2ca02c', alpha=0.8)
    ax.set_title(f'{selected_symbol} Weekly Execution Progress')
    ax.set_ylabel('%')
    ax.set_xlabel('Date')
    ax.legend()
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
